<h1>IRIS_load_data

In [2]:
import pymysql
import pandas as pd

def load_iris_data():
    csv_file = 'iris.csv'
    iris = pd.read_csv(csv_file)

    conn = pymysql.connect(host='localhost', user='june', password='wnsgur97895', db='datascience')
    curs = conn.cursor(pymysql.cursors.DictCursor)
        
    drop_sql = """drop table if exists iris """
    curs.execute(drop_sql)
    conn.commit()
    
    import sqlalchemy
    database_username = 'june'
    database_password = 'wnsgur97895'
    database_ip       = 'localhost'
    database_name     = 'datascience'
    database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.
                                                   format(database_username, database_password, 
                                                          database_ip, database_name))
    iris.to_sql(con=database_connection, name='iris', if_exists='replace')  
    
def load_dbscore_data():
    xl_file = 'db_score.xlsx'
    db_score = pd.read_excel(xl_file)

    conn = pymysql.connect(host='localhost', user='june', password='wnsgur97895', db='datascience')
    curs = conn.cursor(pymysql.cursors.DictCursor)
        
    drop_sql = """drop table if exists db_score """
    curs.execute(drop_sql)
    conn.commit()
    
    import sqlalchemy
    database_username = 'june'
    database_password = 'wnsgur97895'
    database_ip       = 'localhost'
    database_name     = 'datascience'
    database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.
                                                   format(database_username, database_password, 
                                                          database_ip, database_name))
    db_score.to_sql(con=database_connection, name='db_score', if_exists='replace')      

load_iris_data()
print("성공")
'''
if __name__ == '__main__':    
    load_iris_data()
    load_dbscore_data()  
'''

성공


"\nif __name__ == '__main__':    \n    load_iris_data()\n    load_dbscore_data()  \n"

<h1>IRIS_classification

In [6]:
import pymysql
import numpy as np


def classification_performance_eval(y, y_predict):
    tp, tn, fp, fn = 0,0,0,0
    
    for y, yp in zip(y,y_predict):
        if y == 1 and yp == 1:
            tp += 1
        elif y == 1 and yp == -1:
            fn += 1
        elif y == -1 and yp == 1:
            fp += 1
        else:
            tn += 1
            
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = (tp)/(tp+fp)
    recall = (tp)/(tp+fn)
    f1_score = 2*precision*recall / (precision+recall)
    
    return accuracy, precision, recall, f1_score



conn = pymysql.connect(host='localhost', user='june', password='wnsgur97895', db='datascience')
curs = conn.cursor(pymysql.cursors.DictCursor)

sql = "select * from iris"
curs.execute(sql)

data  = curs.fetchall()

#print(data)

curs.close()
conn.close()

X = [ (t['sepal_length'], t['sepal_width'], t['petal_length'], t['petal_width'] ) for t in data ]
X = np.array(X)

y = [ 1 if (t['species'] == 'versicolor') else -1 for t in data]
y = np.array(y)

#print(y)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

from sklearn import tree

dtree = tree.DecisionTreeClassifier()

dtree_model = dtree.fit(X_train, y_train)

y_predict = dtree_model.predict(X_test)

acc, prec, rec, f1 = classification_performance_eval(y_test, y_predict)




print("=======================================")

print("accuracy=%f" %acc)
print("precision=%f" %prec)
print("recall=%f" %rec)
print("f1_score=%f" %f1)

print("=======================================")



from sklearn.model_selection import KFold

accuracy = []
precision = []
recall = []
f1_score = []

kf = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dtree = tree.DecisionTreeClassifier()
    dtree = dtree_model.fit(X_train, y_train)
    y_predict = dtree_model.predict(X_test)
    acc, prec, rec, f1 = classification_performance_eval(y_test, y_predict)
    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)


   
import statistics



print("average_accuracy =", statistics.mean(accuracy))
print("average_precision =", statistics.mean(precision))
print("average_recall =", statistics.mean(recall))
print("average_f1_score =", statistics.mean(f1_score))

print("=======================================")





        

accuracy=0.980000
precision=0.937500
recall=1.000000
f1_score=0.967742
average_accuracy = 0.96
average_precision = 0.9492307692307692
average_recall = 0.9400000000000001
average_f1_score = 0.9411111111111111
